In [40]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns 
import os

# =========================================================
# ⚙️ KONFIGURASI
# =========================================================
FOLDER_DATA = "/work/open_meteo_climate"
NAMA_FILE = "kebumen_75tahun_lengkap.csv" 
PATH_FILE = os.path.join(FOLDER_DATA, NAMA_FILE)

In [43]:
# =========================================================
# 1. LOAD DATA (MEMUAT DATA)
# =========================================================
def load_data(filepath):
    print(f"📂 Mencoba membuka file: {filepath}")
    if not os.path.exists(filepath):
        print(f"❌ File tidak ditemukan di path tersebut.")
        return None
    
    try:
        # Baca CSV, jadikan kolom 'date' sebagai Index Waktu
        df = pd.read_csv(filepath, index_col='date', parse_dates=True)
        print(f"✅ Data berhasil dimuat! Total: {len(df)} baris.")
        
        # Validasi nama kolom (opsional, untuk memastikan kolom ada)
        required_cols = ['temperature', 'rain_mm']
        if not all(col in df.columns for col in required_cols):
            print(f"⚠️ Peringatan: Kolom {required_cols} mungkin tidak ditemukan/berbeda nama.")
            print(f"   Kolom yang ada: {df.columns.tolist()}")
            
        return df
    except Exception as e:
        print(f"❌ Gagal membaca CSV: {e}")
        return None

# =========================================================
# 2. STATISTIK DASAR (DESCRIPTIVE)
# =========================================================
def show_basic_stats(df):
    print("\n--- 📊 STATISTIK DASAR ---")
    # Hanya ambil kolom numerik untuk statistik
    stats = df[['temperature', 'rain_mm']].describe().T 
    print(stats[['mean', 'min', 'max', 'std']])
    
    # Total Hujan (Akumulasi) dalam dataset
    total_hujan = df['rain_mm'].sum()
    print(f"\n🌧️ Total Curah Hujan (Seluruh Dataset): {total_hujan:.2f} mm")

# =========================================================
# 3. FUNGSI PLOTTING
# =========================================================
def plot_periode_tertentu(df, target_waktu):
    """
    Memplot data cuaca berdasarkan filter waktu tertentu.
    """
    print(f"\n🔍 Mencari data untuk periode: {target_waktu}...")
    
    try:
        # --- SLICING (FILTER DATA) ---
        # Menggunakan partial string indexing pandas
        subset = df.loc[target_waktu]
        
        if subset.empty:
            print("❌ Data kosong! Tanggal tersebut tidak ada dalam dataset.")
            return

        # --- SETUP PLOT ---
        fig, ax1 = plt.subplots(figsize=(12, 6))
        
        # --- PLOT SUHU (GARIS) ---
        ax1.plot(subset.index, subset['temperature'], 
                 color='darkorange', marker='.', linestyle='-', linewidth=1.5, label='Suhu (°C)')
        ax1.set_ylabel('Suhu (°C)', color='darkorange', fontweight='bold')
        ax1.tick_params(axis='y', labelcolor='darkorange')
        ax1.grid(True, linestyle='--', alpha=0.5)
        
        # --- PLOT HUJAN (BATANG) ---
        ax2 = ax1.twinx()
        
        # Logika lebar batang: Harian lebih lebar, Bulanan/Tahunan lebih tipis
        # Jika panjang string target > 7 (misal "2022-10-20"), berarti Harian
        width = 0.04 if len(str(target_waktu)) > 7 else 0.5 
        
        ax2.bar(subset.index, subset['rain_mm'], 
                color='dodgerblue', alpha=0.4, width=width, label='Hujan (mm)')
        ax2.set_ylabel('Hujan (mm)', color='dodgerblue', fontweight='bold')
        ax2.tick_params(axis='y', labelcolor='dodgerblue')
        
        # --- FORMAT SUMBU X (WAKTU) OTOMATIS ---
        if len(str(target_waktu)) > 7: # Format Harian (YYYY-MM-DD)
            ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
            ax1.set_xlabel('Jam (WIB)')
            judul = f"Cuaca Detail Tanggal: {target_waktu}"
        elif len(str(target_waktu)) == 7: # Format Bulanan (YYYY-MM)
            ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b'))
            ax1.set_xlabel('Tanggal')
            judul = f"Tren Cuaca Bulan: {target_waktu}"
        else: # Format Tahunan (YYYY)
            ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
            ax1.set_xlabel('Bulan')
            judul = f"Tren Cuaca Tahun: {target_waktu}"

        plt.title(judul, fontweight='bold', fontsize=14)
        
        # Menggabungkan legend
        lines_1, labels_1 = ax1.get_legend_handles_labels()
        lines_2, labels_2 = ax2.get_legend_handles_labels()
        ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper left')
        
        plt.tight_layout()
        plt.show()
        
    except KeyError:
        print(f"❌ Format tanggal salah atau data tidak ditemukan untuk: {target_waktu}")
    except Exception as e:
        print(f"⚠️ Terjadi error saat plotting: {e}")

In [55]:
# =========================================================
# EKSEKUSI UTAMA (MAIN)
# =========================================================
if __name__ == "__main__":
    # 1. Load Data Sekali Saja
    df_cuaca = load_data(PATH_FILE)
    
    # Pastikan data berhasil dimuat sebelum lanjut
    if df_cuaca is not None:
        
        # 2. Tampilkan Statistik Umum
        show_basic_stats(df_cuaca)
        
        # 3. Visualisasi Sesuai Request
        print("\n--- 🎨 MULAI VISUALISASI ---")
        
        # Contoh 1: Lihat Detail SATU HARI (24 Jam)
        # Pastikan tanggal ini ada di data CSV kamu
        plot_periode_tertentu(df_cuaca, "2022-10-22")

        # Contoh 2: Lihat Tren SATU BULAN (30 Hari)
        plot_periode_tertentu(df_cuaca, "2022-10")

        # Contoh 3: Lihat Tren SATU TAHUN (12 Bulan)
        plot_periode_tertentu(df_cuaca, "2022")

📂 Mencoba membuka file: /work/open_meteo_climate/kebumen_75tahun_lengkap.csv
✅ Data berhasil dimuat! Total: 665472 baris.

--- 📊 STATISTIK DASAR ---
                  mean     min     max       std
temperature  25.076715  15.705  33.755  2.212932
rain_mm       0.288467   0.000  46.600  0.884898

🌧️ Total Curah Hujan (Seluruh Dataset): 191967.00 mm

--- 🎨 MULAI VISUALISASI ---

🔍 Mencari data untuk periode: 2022-10-22...
❌ Format tanggal salah atau data tidak ditemukan untuk: 2022-10-22

🔍 Mencari data untuk periode: 2022-10...
❌ Format tanggal salah atau data tidak ditemukan untuk: 2022-10

🔍 Mencari data untuk periode: 2022...
❌ Format tanggal salah atau data tidak ditemukan untuk: 2022


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=672c3bf7-b636-46e8-b7c0-4f6feec29313' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>